In [99]:
%matplotlib inline
import seaborn as sns
import pandas as pd
from os import listdir

all_files = []
for file in os.listdir("./twitter_data"):
    if file.endswith(".csv"):
        all_files.append(os.path.join("./twitter_data", file))

print(all_files)
all_tweets_df = pd.concat((pd.read_csv(f) for f in all_files)).sort_values(by=['username', 'date'])
print(all_tweets_df[['username']].nunique())
all_tweets_df.sample(20)

['./twitter_data/gotracz.csv', './twitter_data/katarzynaobara.csv', './twitter_data/martalempart.csv', './twitter_data/michalakjerzy.csv', './twitter_data/rozeckapl.csv', './twitter_data/sutrykjacek.csv']
username    6
dtype: int64


,date,time,username,tweet,link,likes,retweets,replies,mentions,hashtags
117,2018-09-15,19:17:45,MichalakJerzy,Na Ołbinie zwiększymy środki na wymianę staryc...,https://twitter.com/MichalakJerzy/status/10410...,28,9,1,NaN,"#IdzieNowe,#Michalak2018"
960,2018-06-14,13:11:25,martalempart,@kongreskobiet @MartaNiewierko może się wreszc...,https://twitter.com/martalempart/status/100721...,0,0,1,"MartaNiewierko,kongreskobiet",NaN
646,2018-07-20,16:20:44,martalempart,@moanrosa odzieracie obywatelki i obywatelki z...,https://twitter.com/martalempart/status/102031...,17,9,3,moanrosa,NaN
695,2018-07-18,12:25:54,martalempart,Mobilizacja dziś pod Sejmem o 17.00! Podajcie ...,https://twitter.com/martalempart/status/101952...,78,60,2,NaN,#SejmJestNasz
12,2018-10-06,15:09:22,GoTracz,Zieloni gotowi ma 10. Wrocławski Marsz Równośc...,https://twitter.com/GoTracz/status/10485608493...,40,7,3,NaN,NaN
89,2018-04-22,15:40:43,GoTracz,Niedzielne przedpołudnie upłynęło na bardzo mi...,https://twitter.com/GoTracz/status/98804996619...,13,3,4,NaN,NaN
37,2018-10-15,13:06:15,SutrykJacek,Zaczynamy #briefing #programdlaWrocławia #rodz...,https://twitter.com/SutrykJacek/status/1051791...,7,2,0,NaN,"#briefing,#programdlaWrocławia,#rodzina,#pokol..."
163,2018-09-15,12:21:46,SutrykJacek,Cały #Wrocław musi być miastem bliżej: bliżej ...,https://twitter.com/SutrykJacek/status/1040908...,43,14,3,NaN,#Wrocław
630,2018-07-20,17:01:36,martalempart,"@bbudka o Dudzie, który oszukał Polaków, udają...",https://twitter.com/martalempart/status/102032...,2,1,1,bbudka,NaN
184,2018-09-08,10:42:33,SutrykJacek,Za moment zaczynamy Konwencję #KoalicjaObywate...,https://twitter.com/SutrykJacek/status/1038346...,34,17,0,NaN,#KoalicjaObywatelska
